DIY Covid-19 Dashboard Li Wang (l.wang@se20.qmul.ac.uk Student Number: 200149271) All rights reserved.

# DIY Covid-19 Dashboard


NewCases:  New Cases By Publish Date

CurriedTests: :New Tests By Publish Date

PlanedTested : Planned Capacity By Publish Date

In [49]:
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [50]:
## Get data from API
def updatedata1():
    filters1= [
        'areaType=overview'
    ]
    structure1= {
         "date":"date",
         "NewCases":"newCasesByPublishDate",
         "CurriedTests":"newTestsByPublishDate",
         "PlanedTested":"plannedCapacityByPublishDate"
          }
    api2 = Cov19API (filters=filters1, structure=structure1)
    testsdistribution=api2.get_json()
    with open("testsdistribution.json", "wt") as OUTF:
         json.dump(testsdistribution, OUTF)

In [51]:
## Wrangle Data
def parse_date1 (datestring):
    return pd.to_datetime(datestring, format="%Y-%m-%d")

def wrangledata1():
    with open("testsdistribution.json", "rt") as INFIFLE1:
         data1 =json.load(INFIFLE1)
    datalist1 =data1['data']
    dates1 = [dictionary ['date'] for dictionary in datalist1 ]
    dates1.sort()
    startdate1 =parse_date1(dates1[0])
    enddate1 =parse_date1(dates1[-1])
    index1 =pd.date_range(startdate1, enddate1, freq='D')
    timeseriesdf1=pd.DataFrame(index=index1, columns=['NewCases','CurriedTests', 'PlanedTested'])
    for entry in datalist1: 
        dates1=parse_date1(entry['date'])
        for column in ['NewCases','CurriedTests', 'PlanedTested']:
            if pd.isna(timeseriesdf1.loc[dates1, column]): 
                value1= float(entry[column]) if entry[column]!=None else 0.0
                timeseriesdf1.loc[dates1, column]=value1
    timeseriesdf1.fillna(0.0, inplace=True)
    timeseriesdf1.to_pickle("timeseriesdf1.pkl")

In [52]:
##Graphs
series1=wdg.SelectMultiple(
    options=['NewCases', 'CurriedTests', 'PlanedTested'],
    value=['NewCases', 'CurriedTests', 'PlanedTested'],
    rows=3,
    description='Stats:',
    disabled=False
)
scale1=wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
)
apibutton1=wdg.Button(
    description='update data',
    disabled=False,
    button_style='info', 
    tooltip='Click to download current Public Health England data',
    icon='refresh' 
)
controls1=wdg.HBox([series1, scale1,apibutton1])

In [53]:
def timeseries1_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        timeseriesdf1[list(gcols)].plot(logy=logscale)
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

In [54]:
##update data by press the button
def access_api1(button):
    updatedata1()
    wrangledata1()
    global timeseriesdf1
    timeseriesdf1=pd.read_pickle("timeseriesdf1.pkl")
    current=scale1.value
    if current==scale1.options[0]:
        other=scale1.options[1]
    else:
        other=scale1.options[0]
    scale1.value=other 
    scale1.value=current 

In [55]:
## Graphs and Analysis
apibutton1.on_click(access_api1) 
wrangledata1()
timeseriesdf1=pd.read_pickle("timeseriesdf1.pkl")
graph=wdg.interactive_output(timeseries1_graph, {'gcols': series1, 'gscale': scale1})
display(controls1, graph)

Output()

The data is up to date until 2020-11-27 00:00:00


**Data source: *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*

**DIY Covid-19 Dashboard Kit Resource: (https://github.com/fsmeraldi/diy-covid19dash) (C) , 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). All rights reserved.*
